# Analisis Mediasi dengan package seminr
### Langkah utama untuk analisis PLS-SEM yaitu:
 - A. Persiapan ekosistem, data dan model,
 - B. Measurement Model Assessment (periksa indikator ke variabel),
 - C. Structural Model Assessment (periksa antar variabel),
 - D. uji Moderasi / Mediasi
 - E. Plot gambar hasil analisis seminr

### A. Persiapan ekosistem, data dan model,

In [ ]:
# a1. instal library seminr jika belum ada
# install.packages("seminr")

In [25]:
# Make the SEMinR library ready to use
library("seminr")

Gambar model PLS SEM yang akan diuji

<img src="corporate case.png" alt="Alt text" width="600px" />

video mengenai **PLS SEM dengan seminr** di channel YouTube kami

<iframe width="560" height="315" src="https://www.youtube.com/embed/QIKLTsvOZLg?si=hQry8N1Myd8A4hpJ" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [26]:
# a2. Load the corporate reputation data
corp_rep_data <- read.csv(file = "Corporate Reputation Data.csv", header = TRUE, sep = ";")

# Show the first several rows of the corporate reputation data
# head(corp_rep_data)

In [27]:
# a3. Create measurement model 
# gambar model di link 
simple_mm <- constructs(
  composite("COMP", multi_items("comp_", 1:3)), #competence
  composite("LIKE", multi_items("like_", 1:3)), #Likeability
  composite("CUSA", single_item("cusa")), #customer satisfaction
  composite("CUSL", multi_items("cusl_", 1:3)) #customer loyalty
)

In [28]:
# a4. Create structural model 
simple_sm <- relationships(
  paths(to = c("CUSA","CUSL"), from = c("COMP", "LIKE")), 
  paths(to = "CUSL", from = c("CUSA"))
)

In [29]:
# a5. Estimate the model
# Estimate the model with default settings
corp_pls <- estimate_pls(data = corp_rep_data, 
    measurement_model = simple_mm,
    structural_model = simple_sm,
    missing_value = "-99")

Generating the seminr model
All 344 observations are valid.


In [30]:
sum_corp_pls <- summary(corp_pls)
sum_corp_pls


Results from  package seminr (2.3.7)

Path Coefficients:
        CUSA  CUSL
R^2    0.295 0.562
AdjR^2 0.290 0.558
COMP   0.162 0.009
LIKE   0.424 0.342
CUSA       . 0.504

Reliability:
     alpha  rhoC   AVE  rhoA
COMP 0.776 0.865 0.681 0.832
LIKE 0.831 0.899 0.747 0.836
CUSA 1.000 1.000 1.000 1.000
CUSL 0.831 0.899 0.748 0.839

Alpha, rhoC, and rhoA should exceed 0.7 while AVE should exceed 0.5


### B dan C
Langkah selanjutnya adalah 

 - B. Measurement Model Assessment (periksa indikator ke variabel),
 - C. Structural Model Assessment (periksa antar variabel),

detail prosesnya langkah B dan C bisa lihat video mengenai **PLS SEM dengan seminr** di channel YouTube kami

<iframe width="560" height="315" src="https://www.youtube.com/embed/QIKLTsvOZLg?si=hQry8N1Myd8A4hpJ" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [31]:
# 7. Bootstrap untuk inference (SE, CI, p-value)
set.seed(123)
corp_boot <- bootstrap_model(corp_pls, nboot = 1000)

Bootstrapping model using seminr...
SEMinR Model successfully bootstrapped


In [32]:
# 8. Ringkasan bootstrap (koef direct & interaksi, CI, p-value)
corp_boot_sum <- summary(corp_boot, alpha = 0.05)

In [33]:
# Inspect the bootstrapped structural paths
corp_boot_sum$bootstrapped_paths

               Original Est. Bootstrap Mean Bootstrap SD T Stat. 2.5% CI
COMP  ->  CUSA         0.162          0.163        0.065   2.483   0.037
COMP  ->  CUSL         0.009          0.015        0.054   0.171  -0.092
LIKE  ->  CUSA         0.424          0.423        0.060   7.109   0.308
LIKE  ->  CUSL         0.342          0.339        0.057   6.019   0.227
CUSA  ->  CUSL         0.504          0.504        0.044  11.588   0.418
               97.5% CI
COMP  ->  CUSA    0.287
COMP  ->  CUSL    0.120
LIKE  ->  CUSA    0.541
LIKE  ->  CUSL    0.449
CUSA  ->  CUSL    0.585

Prinsip pengambilan keputusan Mediasi PLS SEM yang akan diuji (Hair et al. 2021)

<img src="mediasi2.png" alt="Alt text" width="600px" />

### Tahap 1

Kita jalankan baris perintah di bawah ini untuk menguji indirect effect variabel COMP-CUSA-CUSL,

In [34]:
# Inspect total indirect effects
sum_corp_pls$total_indirect_effects

      COMP  LIKE  CUSA  CUSL
COMP 0.000 0.000 0.000 0.082
LIKE 0.000 0.000 0.000 0.214
CUSA 0.000 0.000 0.000 0.000
CUSL 0.000 0.000 0.000 0.000

In [35]:

# Inspect indirect effects
specific_effect_significance(corp_boot,
  from = "COMP",
  through = "CUSA",
  to = "CUSL",
  alpha = 0.05
)

 Original Est. Bootstrap Mean   Bootstrap SD        T Stat.        2.5% CI 
    0.08174938     0.08256306     0.03430492     2.38302212     0.01783308 
      97.5% CI 
    0.15070282 

If the indirect effect is not significant, we conclude that CUSA does not function as a mediator in the tested relationship.

The results show that the total indirect effect of COMP on CUSL is 0.0825 and significant, CI [0.0178 : 0.1507] (no zero value between the CI range), we can declare **CUSA is a mediator between COMP and CUSL**.

Gambar hasil uji COMP - CUSA - CUSL

<img src="slide4.png" alt="Alt text" width="600px" />

In [36]:
specific_effect_significance(corp_boot,
  from = "LIKE",
  through = "CUSA",
  to = "CUSL",
  alpha = 0.05
)

 Original Est. Bootstrap Mean   Bootstrap SD        T Stat.        2.5% CI 
    0.21382794     0.21333354     0.03611266     5.92113534     0.14592345 
      97.5% CI 
    0.28572651 

, and the total indirect effect of LIKE on CUSL is 0.2133, and significant, CI [0.1459 : 0.2857] (no zero value between the CI range), we can declare **CUSA is a mediator between LIKE and CUSL**.

Gambar hasil uji LIKE - CUSA - CUSL

<img src="slide6.png" alt="Alt text" width="600px" />

### Tahap 2 menguji direct effects

In [37]:
# Inspect the direct effects
sum_corp_pls$paths

        CUSA  CUSL
R^2    0.295 0.562
AdjR^2 0.290 0.558
COMP   0.162 0.009
LIKE   0.424 0.342
CUSA       . 0.504

In [38]:
# Inspect the confidence intervals for direct effects
corp_boot_sum$bootstrapped_paths

               Original Est. Bootstrap Mean Bootstrap SD T Stat. 2.5% CI
COMP  ->  CUSA         0.162          0.163        0.065   2.483   0.037
COMP  ->  CUSL         0.009          0.015        0.054   0.171  -0.092
LIKE  ->  CUSA         0.424          0.423        0.060   7.109   0.308
LIKE  ->  CUSL         0.342          0.339        0.057   6.019   0.227
CUSA  ->  CUSL         0.504          0.504        0.044  11.588   0.418
               97.5% CI
COMP  ->  CUSA    0.287
COMP  ->  CUSL    0.120
LIKE  ->  CUSA    0.541
LIKE  ->  CUSL    0.449
CUSA  ->  CUSL    0.585

The results show that the direct effect from COMP to CUSL is 0.015 with a 95% confidence interval [−0.092; 0.120]. As this interval includes zero, this direct effect is not significant. According to the guidelines in Hair et al., we therefore conclude that the relationship between COMP and CUSL is **fully mediated** by CUSA.

<img src="slide5.png" alt="Alt text" width="600px" />

Next, we need to consider the direct relationship between LIKE and CUSL, which has a 0.339 path coefficient with a 95%
confidence interval [0.227; 0.449]. As this confidence interval does not include zero, we conclude that CUSA **partially mediates** the effect of LIKE on CUSL.

<img src="slide7.png" alt="Alt text" width="600px" />

In [39]:
# Calculate the sign of p1*p2*p3
sum_corp_pls$paths["LIKE", "CUSL"] * sum_corp_pls$paths["LIKE","CUSA"] * sum_corp_pls$paths["CUSA","CUSL"]

[1] 0.073144

In [ ]:
# Inspect the bootstrapped indicator loadings 
corp_boot_sum$bootstrapped_loadings

# d1. Pastikan paket terpasang jika belum ada
# install.packages(c("DiagrammeR", "DiagrammeRsvg", "rsvg"))

library(DiagrammeR)
library(DiagrammeRsvg)
library(rsvg)

# d2. Buat atau tangkap graph HTMLwidget (grViz)
graph <- plot(corp_boot, title = "Bootstrapped Model")

# d3. Ekspor ke SVG string
svg_graph <- export_svg(graph)
nama_file <- "bootstrap_model_hd modiation.png"
# d4. Render ke PNG 1920×1080 px dan simpan ke file .png
rsvg_png(
  svg    = charToRaw(svg_graph),      # raw vector SVG
  file   = nama_file,  # nama file output
  width  = 1920,                      # lebar piksel
  height = 1080                       # tinggi piksel
)

message(sprintf("Gambar model sudah tersimpan di folder dengan nama %s", nama_file))

## Variable

Competence (COMP)
comp_1 [The company] is a top competitor in its market
comp_2 As far as I know, [the company] is recognized worldwide
comp_3 I believe [the company] performs at a premium level

Likeability (LIKE)
like_1 [The company] is a company I can better identify with than other companies
like_2 [The company] is a company I would regret more not having if it no longer existed than I would other companies
like_3 I regard [the company] as a likeable company

Customer satisfaction (CUSA)
cusa I am satisfied with [the company]

Customer loyalty (CUSL)
cusl_1 I would recommend [company] to friends and relatives
cusl_2 If I had to choose again, I would choose [company] as my mobile phone service provider
cusl_3 I will remain a customer of [company] in the future

Quality (QUAL)
qual_1 The products/services offered by [the company] are of high quality
qual_2 [The company] is an innovator, rather than an imitator with respect to [industry]
qual_3 [The company]’s products/services offer good value for money
qual_4 The products/services offered by [the company] are good
qual_5 Customer concerns are held in high regard at [the company]
qual_6 [The company] is a reliable partner for customers
qual_7 [The company] is a trustworthy company
qual_8 I have a lot of respect for [the company]

Performance (PERF)
perf_1 [The company] is a very well-managed company
perf_2 [The company] is an economically stable company
perf_3 The business risk for [the company] is modest compared to its competitors
perf_4 [The company] has growth potential
perf_5 [The company] has a clear vision about the future of the company

Corporate social responsibility (CSOR)
csor_1 [The company] behaves in a socially conscious way
csor_2 [The company] is forthright in giving information to the public
csor_3 [The company] has a fair attitude toward competitors
csor_4 [The company] is concerned about the preservation of the environment
csor_5 [The company] is not only concerned about profits

Attractiveness (ATTR)
attr_1 [The company] is successful in attracting high-quality employees
attr_2 I could see myself working at [the company]
attr_3 I like the physical appearance of [the company] (company, buildings, shops, etc.)

Switching costs indicators (SC)
switch_1 It takes me a great deal of time to switch to another company
switch_2 It costs me too much to switch to another company
switch_3 It takes a lot of effort to get used to a new company with its specific "rules” and practices
switch_4 In general, it would be a hassle switching to another company

Kita akan coba moderasi SC dan CSOR ke CUSA

Single-item measures of QUAL, PERF, CSOR, and ATTR for the redundancy analysis
qual_global Please assess the overall quality of [the company’s] activities
perf_global Please assess [the company’s] overall performance
csor_global Please assess the extent to which [the company] acts in socially conscious ways
attr_global [The company] has a high overall attractiveness

# Reference
 - Cohen, J. (1988). Statistical power analysis for the behavioral sciences. Mahwah, NJ: Lawrence Erlbaum.
 - data Corporate: https://www.pls-sem.net/downloads/pls-sem-using-r-a-workbook/
 - Fornell, C., & Larcker, D. F. (1981). Evaluating structural equation models with unobservable variables and measurement error. Journal of Marketing Research, 18(1), 39–50.
 - Hair, J. F., Hult, G. T. M., Ringle, C., & Sarstedt, M. (2017). A primer on partial least squares structural equation modeling (PLS-SEM). In SAGE Publications,
 - Hair, J. F., Risher, J. J., Sarstedt, M., & Ringle, C. M. (2019). When to use and how to report the results of PLS-SEM. European Business Review, 31(1), 2–24.
 - Hair et al. 2021, https://link.springer.com/book/10.1007/978-3-030-80519-7
 - Henseler, J., Ringle, C. M., & Sarstedt, M. (2015). A new criterion for assessing discriminant validity in variance-based structural equation modeling. Journal of the Academy of Marketing Science, 43(1), 115–135.
 - https://mirror.twds.com.tw/CRAN/web/packages/seminr/seminr.pdf
 - https://github.com/sem-in-r/seminr/blob/master/vignettes/SEMinR.Rmd
 - https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3900621
 - https://cran.r-project.org/web/packages/seminr/index.html